## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import csv
import os, sys
currentdir = os.path.dirname(os.path.realpath("Vacation_Search"))
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_Database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv", index_col=0)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Punta Arenas,CL,-53.15,-70.92,51.01,68,61,26.40,broken clouds
1,Saint-Joseph,RE,-21.37,55.62,77.00,73,0,5.82,clear sky
2,Port Alfred,ZA,-33.59,26.89,60.01,86,0,1.01,clear sky
3,Marystown,CA,47.17,-55.15,33.31,91,0,10.09,clear sky
4,San Julian,MX,21.02,-102.17,59.67,30,2,8.84,clear sky
5,Kuching,MY,1.55,110.33,78.80,88,20,0.34,few clouds
6,Tuktoyaktuk,CA,69.45,-133.04,-0.40,84,90,3.36,light snow
7,Mount Gambier,AU,-37.83,140.77,62.60,55,90,12.75,overcast clouds
8,Vila Franca Do Campo,PT,37.72,-25.43,55.40,62,20,17.22,light rain
9,Chalmette,US,29.94,-89.96,46.40,31,1,8.05,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 8
What is the maximum temperature you would like for your trip? 56


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
vacation_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
vacation_cities_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Punta Arenas,CL,-53.15,-70.92,51.01,68,61,26.40,broken clouds
3,Marystown,CA,47.17,-55.15,33.31,91,0,10.09,clear sky
8,Vila Franca Do Campo,PT,37.72,-25.43,55.40,62,20,17.22,light rain
9,Chalmette,US,29.94,-89.96,46.40,31,1,8.05,clear sky
11,Albany,US,42.60,-73.97,54.00,88,100,4.00,moderate rain
13,Husavik,IS,66.04,-17.34,48.20,61,75,26.40,broken clouds
14,Bay Roberts,CA,47.60,-53.26,30.99,63,75,12.75,broken clouds
18,Kuopio,FI,62.89,27.68,32.00,97,90,9.17,light snow
25,Orbetello,IT,42.44,11.22,55.40,93,40,3.36,scattered clouds
28,Ushuaia,AR,-54.80,-68.30,53.60,30,40,17.22,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
vacation_cities_df.count()

City                   247
Country                247
Lat                    247
Lng                    247
Max Temp               247
Humidity               247
Cloudiness             247
Wind Speed             247
Current Description    247
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = vacation_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Punta Arenas,CL,51.01,broken clouds,-53.15,-70.92,
3,Marystown,CA,33.31,clear sky,47.17,-55.15,
8,Vila Franca Do Campo,PT,55.40,light rain,37.72,-25.43,
9,Chalmette,US,46.40,clear sky,29.94,-89.96,
11,Albany,US,54.00,moderate rain,42.60,-73.97,
13,Husavik,IS,48.20,broken clouds,66.04,-17.34,
14,Bay Roberts,CA,30.99,broken clouds,47.60,-53.26,
18,Kuopio,FI,32.00,light snow,62.89,27.68,
25,Orbetello,IT,55.40,scattered clouds,42.44,11.22,
28,Ushuaia,AR,53.60,scattered clouds,-54.80,-68.30,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
Lat = row["Lat"]
Lng = row["Lng"]
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
    "location": f"{Lat},{Lng}"
}
    Lat = row["Lat"]
    Lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError, NameError):
        print("Hotel not found ... skip")
        

Hotel not found ... skip
Hotel not found ... skip
Hotel not found ... skip
Hotel not found ... skip
Hotel not found ... skip
Hotel not found ... skip
Hotel not found ... skip
Hotel not found ... skip
Hotel not found ... skip
Hotel not found ... skip
Hotel not found ... skip
Hotel not found ... skip
Hotel not found ... skip
Hotel not found ... skip
Hotel not found ... skip
Hotel not found ... skip
Hotel not found ... skip
Hotel not found ... skip
Hotel not found ... skip
Hotel not found ... skip
Hotel not found ... skip
Hotel not found ... skip


In [76]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df[hotel_df["Hotel Name"] != ""]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Punta Arenas,CL,51.01,broken clouds,-53.15,-70.92,Hotel Dreams Del Estrecho
3,Marystown,CA,33.31,clear sky,47.17,-55.15,Hotel Dreams Del Estrecho
8,Vila Franca Do Campo,PT,55.40,light rain,37.72,-25.43,Marystown Hotel & Convention Centre
9,Chalmette,US,46.40,clear sky,29.94,-89.96,Pestana Bahia Praia
11,Albany,US,54.00,moderate rain,42.60,-73.97,Marina Motel Chalmette
14,Bay Roberts,CA,30.99,broken clouds,47.60,-53.26,Bjarnabúð
18,Kuopio,FI,32.00,light snow,62.89,27.68,Stonewall Inn
25,Orbetello,IT,55.40,scattered clouds,42.44,11.22,Hotel Jahtihovi
28,Ushuaia,AR,53.60,scattered clouds,-54.80,-68.30,Argentario Golf Resort & Spa
31,Semnan,IR,37.89,light rain,35.57,53.40,Albatros Hotel


In [77]:
# 8a. Create the output File (CSV)
output_data_file = "clean_hotel_df.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [84]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp}°F </dd>
"""


# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
clean_hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [87]:
# 11a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1)
marker_layer = gmaps.marker_layer(locations, info_box_content=clean_hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))